# Harmonization of MIDRC Imaging Study Descriptions using the LOINC Playbook
---
by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

November 2022

---
This Jupyter notebook demonstrates how to map the imaging_study node's `study_description` and `modality` properties to a `loinc_code` by utilizing the MIDRC DQH committee's [LOINC mapping table in GitHub](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out). Once the imaging study has been mapped to a LOINC code, the code can be used to derive other LOINC properties. The following [LOINC properties](https://github.com/uc-cdis/midrc_dictionary/blob/deae581f0fb8b9ae5add1458d7882e189ba97af6/gdcdictionary/schemas/imaging_study.yaml#L77) are on the `imaging_study` node of the [MIDRC data dictionary](https://data.midrc.org/dd):
* loinc_code
* loinc_long_common_name
* loinc_method
* loinc_system
* loinc_contrast

In [ ]:
# Import Python Packages and scripts
import pandas as pd
import numpy as np
from pathlib import Path
import sys, os, copy, datetime, shutil


import gen3
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query


In [ ]:
# who's running the notebook

username = ""

In [ ]:
# download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
loinc_dir = "/Users/{}/Documents/Notes/MIDRC/LOINC".format(username) #set your working dir
os.chdir(loinc_dir)
os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O expansion.py")
from expansion import Gen3Expansion


In [ ]:
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
sapi = 'https://staging.midrc.org'
scred = '/Users/{}/Downloads/midrc-staging-credentials.json'.format(username)
sauth = Gen3Auth(sapi, refresh_file=scred) # authentication class
ssub = Gen3Submission(sapi, sauth) # submission class
squery = Gen3Query(sauth) # query class
sexp = Gen3Expansion(sapi,sauth,ssub) # class with some custom scripts
sexp.get_project_ids()
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://validatestaging.midrc.org/identity
vsapi = 'https://validatestaging.midrc.org'
vscred = '/Users/{}/Downloads/midrc-validatestaging-credentials.json'.format(username)
vsauth = Gen3Auth(vsapi, refresh_file=vscred) # authentication class
vssub = Gen3Submission(vsapi, vsauth) # submission class
vsquery = Gen3Query(vsauth) # query class
vsexp = Gen3Expansion(vsapi,vsauth,vssub) # class with some custom scripts
vsexp.get_project_ids()


## Prepare the mapping and filter tables
---
Get the mapping table and the filtering attributes tables from the [MIDRC GitHub repository](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out) and reformat them to prepare for the LOINC mapping.

Prior to this, you will need to clone the repo using `git clone git@github.com:MIDRC/midrc_dicom_harmonization.git`


In [ ]:
## Set the directory to your copy of the GitHub repo and pull main branch to pull latest updates from GitHub
git_dir = "/Users/{}/Documents/GitHub/MIDRC/midrc_dicom_harmonization/".format(username)
os.chdir(git_dir)
os.system("git checkout main")
os.system("git pull origin main")
os.chdir(loinc_dir)


In [ ]:
## Create the output/working directory
now = datetime.datetime.now()
today = "{}-{}-{}".format(now.year, now.month, now.day)

results_dir = "{}/results_{}".format(loinc_dir,today)
Path(results_dir).mkdir(parents=True, exist_ok=True)

staging_results_dir = "{}/staging_results_{}".format(results_dir,today)
Path(staging_results_dir).mkdir(parents=True, exist_ok=True)

validatestaging_results_dir = "{}/validatestaging_results_{}".format(results_dir,today)
Path(validatestaging_results_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
mapping_file = "{}/out/StudyDescription_mapping_table.csv".format(git_dir)
mapping = pd.read_csv(mapping_file,dtype=str) #['Modality', 'StudyDescription', 'LOINC code', 'L-Long Common Name']

shutil.copy2(mapping_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

mapping.rename(columns={"StudyDescription":"study_description","Modality":"study_modality","LOINC code":"loinc_code","L-Long Common Name":"loinc_long_common_name"},inplace=True)
mapping.drop_duplicates(inplace=True)
mapping['study_description'] = mapping.apply(lambda row: row['study_description'].casefold(),axis=1)
mapping['study_modality'] = mapping.apply(lambda row: row['study_modality'].casefold(),axis=1)

## Change any "(blank)" values for study_description to "[blank]" to match the mapping table
mapping.replace({"study_description":{
    '(blank)':'[blank]'}},
    inplace=True)

## remove any leading/trailing whitespaces in the codes/descriptions
mapping['loinc_code'] = mapping['loinc_code'].str.strip()
mapping['loinc_long_common_name'] = mapping['loinc_long_common_name'].str.strip()
mapping['study_description'] = mapping['study_description'].str.strip()
mapping['study_modality'] = mapping['study_modality'].str.strip()

## Summarize the mapping table
modalities = list(set(mapping['study_modality']))
descriptions = list(set(mapping['study_description']))
mapping_codes = list(set(mapping['loinc_code']))
mapping_names = list(set(mapping['loinc_long_common_name']))

display(mapping)
print("study_modality values in mapping table: {}".format(modalities))
print("Number of unique LOINC codes in mapping table: {}".format(len(mapping_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(mapping_names)))


In [ ]:
filters_file = "{}/out/StudyDescription_filtering_attributes.csv".format(git_dir)
filters = pd.read_csv(filters_file,dtype=str) #['LOINC code', 'L-Long Common Name', 'L-Method', 'L-System', 'Rad.Timing']

shutil.copy2(filters_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

filters.rename(columns= {
    'LOINC code':'loinc_code',
    'L-Long Common Name':'loinc_long_common_name',
    'L-Method':'loinc_method',
    'Rad.Timing':'loinc_contrast',
    'MIDRC-System':'loinc_system'
    },
    inplace=True,
    errors='ignore'
)
filters.drop(columns='L-System',inplace=True,errors='ignore')

## remove any leading/trailing whitespaces
filters['loinc_code'] = filters['loinc_code'].str.strip()
filters['loinc_long_common_name'] = filters['loinc_long_common_name'].str.strip()

filter_codes = list(set(filters['loinc_code']))
filter_names = list(set(filters['loinc_long_common_name']))
print("Number of unique LOINC codes in filters table: {}".format(len(filter_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(filter_names)))


missing_codes = set(mapping_codes).difference(set(filter_codes))
missing_names = set(mapping_names).difference(set(filter_names))

print("Number of LOINC codes in mapping table missing from filters table: {}\n\t{}".format(len(missing_codes),missing_codes))
print("Number of LOINC names in mapping table missing from filters table: {}\n\t{}".format(len(missing_names),missing_names))

display(filters)


## Get the imaging_study information from MIDRC Staging and Validate Staging
---
* Use the [`Gen3Expansion.get_node_tsvs()` SDK function](https://github.com/cgmeyer/gen3sdk-python/blob/389e3945482439ace6e4536e6d0e35c6e48de9c9/expansion/expansion.py#L219) to get all the imaging_studies in MIDRC Staging (staging.midrc.org).

* The function `exp.get_node_tsvs()` will return a master dataframe containing all the imaging_study data from the projects specified.

### In this version of the notebook, we're getting five projects: 
Staging:
* Open-R1 (data from RSNA)
* Open-A1 (data from ACR)
* Open-A1_PETAL_REDCORAL (PETAL data from ACR)

ValidateStaging:
* SEQ_Open-R3
* SEQ_Open-A3

In [ ]:
os.chdir(loinc_dir)

sprojects = ['Open-R1','Open-A1','Open-A1_PETAL_REDCORAL']
sst = sexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=sprojects,outdir=staging_results_dir)

vsprojects = ['SEQ_Open-R3','SEQ_Open-A3']
vst = vsexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=vsprojects,outdir=validatestaging_results_dir)

projects = sprojects + vsprojects

In [ ]:
# concat the seq and open master imaging studies TSVs

st = pd.concat([sst,vst], ignore_index=False)
assert(len(st) == (len(sst)+len(vst)))

In [ ]:
dupes = st.loc[st.duplicated(subset='submitter_id',keep=False)].sort_values(by='submitter_id')
print("There are {} duplicate submitter_ids in the list of imaging studies from Staging and ValidateStaging.".format(len(dupes)))
if len(dupes) > 0:
    display(dupes)


## Reformat the imaging_study data to prepare for mapping
---
* **NaN values**: In the mapping table, a `study_description` or `study_modality` with a value of `null` or `NaN` is listed as `[blank]`.
* **Case-insensitivity**: we should ignore capitalization of `study_modality` and `study_description` strings when we do the look-up. We'll use `str.casefold()` to ignore case for matching.
* **CR/DX Modality**: in the mapping table, all versions of "CR" and "DX" modalities are denoted "CR, DX"; so, we need to change all occurrences of other spellings to match that exactly.
* **Duplicated study UIDs**: Identify any duplicated study UIDs and remove them from studies.


In [ ]:
## Make a copy of the master imaging_study table with only data relevant to LOINC mapping, and change NaN to "[blank]"
studies = copy.deepcopy(st[['submitter_id','study_description','study_modality']])
#studies.index = studies['submitter_id']
studies['study_description'].fillna("[blank]", inplace=True)
studies['study_modality'].fillna("[blank]", inplace=True)

## Make the mapping case-insensitive
studies['study_description'] = studies['study_description'].str.casefold()
studies['study_modality'] = studies['study_modality'].str.casefold()

## remove any leading/trailing whitespaces
studies['study_description'] = studies['study_description'].str.strip()
studies['study_modality'] = studies['study_modality'].str.strip()


## Fix any CR/DX study_modality to all be "cr, dx" to match mapping table
studies.replace({"study_modality":{
    'cr':'cr, dx',
    'dx':'cr, dx',
    'cr,dx':'cr, dx',
    'dx,cr':'cr, dx',
    'dx, cr':'cr, dx',
    'crdx':'cr, dx',
    'dxcr':'cr, dx'}},
    inplace=True)

## Fix any CT/PT study_modality to all be "pt, ct" to match mapping table
studies.replace({"study_modality":{
    'pt':'pt, ctpt',
    'ctpt':'pt, ctpt',
    'ptct':'pt, ctpt',
    'ct, pt':'pt, ctpt',
    'pt, ct':'pt, ctpt',
    'pt,ct':'pt, ctpt',
    'ct,pt':'pt, ctpt'}},
    inplace=True)

## Change any "<NONE>" values for study_description to "[blank]" to match the mapping table
studies.replace({"study_description":{
    '<none>':'[blank]'}},
    inplace=True)

## Identify and drop duplicates
dupes = studies.loc[studies.duplicated(keep=False)]
studies = studies.drop_duplicates()
display(studies)



In [ ]:
## Check modalities for presence in the mapping table and frequency in MIDRC data:
study_modalities = list(set(studies.study_modality))
print("There are the following values of study_modality in the MIDRC imaging_study data: \n{}".format(study_modalities))

missing_modalities = set(study_modalities).difference(modalities)

mfreq = {}
for mod in missing_modalities:
    freq = len(studies.loc[studies['study_modality']==mod])
    mfreq[mod] = freq

print("\nThese modalities are in the study data but not in the mapping table: \n{}".format(mfreq))

## Do mapping using pandas pd.merge(): 
---

1) Merge `loinc_code` in `mapping` dataframe into the `studies` dataframe on the combination of `study_description` and `study_modality`.

2) Merge the `loinc_long_common_name`, `loinc_method`, `loinc_contrast`, and `loinc_system` in `filters` dataframe into the `studies` dataframe on `loinc_code`.



In [ ]:
sdf = studies.merge(mapping,on=['study_modality','study_description'],how='left').drop_duplicates()
sdf = sdf.merge(filters,on=['loinc_code','loinc_long_common_name'],how='left')
sdf.drop(columns=['study_description','study_modality'],inplace=True,errors='ignore')
sdf

In [ ]:
## Pull in the original study_modality and study_description (non-lowercase) and rearrange the columns
cols = ['type','project_id','submitter_id','cases.submitter_id','study_modality','study_description','loinc_code','loinc_long_common_name','loinc_method','loinc_contrast','loinc_system']
sdf = sdf.merge(st[['type','project_id','submitter_id','cases.submitter_id','study_description','study_modality']], on='submitter_id',how='left')[cols]
sdf


In [ ]:
mdf = sdf.loc[~sdf['loinc_code'].isna()]
udf = sdf.loc[sdf['loinc_code'].isna()]
fdf = udf.groupby(["study_modality", "study_description"]).size().reset_index(name="Freq").sort_values(by='Freq',ascending=False)
print("Total Studies: {}, Mapped: {}, Unmapped: {}".format(len(sdf),len(mdf),len(udf)))

In [ ]:
## Save results to files
filename="{}/LOINC_mapping_results_{}_{}.tsv".format(results_dir,len(sdf),today)
sdf.to_csv(filename, sep='\t', index=False)
print("All LOINC mapping results for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/LOINC_mapped_{}_{}.tsv".format(results_dir,len(mdf),today)
mdf.to_csv(filename, sep='\t', index=False)
print("Successfully mapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/LOINC_umapped_{}_{}.tsv".format(results_dir,len(udf),today)
udf.to_csv(filename, sep='\t', index=False)
print("Unmapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/LOINC_umapped_frequencies_{}_{}.tsv".format(results_dir,len(fdf),today)
fdf.to_csv(filename, sep='\t', index=False)
print("Frequencies of unmapped imaging study description/modality combinations for projects {} saved to file: \n{}\n".format(projects,filename))


In [ ]:
## Take a glance at frequency of the unmapped imaging study modality/description combinations
display(fdf)

## Submit the data to Staging and ValidateStaging
---
Use the Gen3SDK function `Gen3Submission.submit_file()` to update the LOINC properties for each of the newly mapped studies in `mdf`.

In [ ]:
# let's first seperate the mapped studies in mdf into open and seq

# open
smdf = mdf.loc[mdf.project_id.isin(sprojects)].reset_index(drop=True)

# sequestered
vsmdf = mdf.loc[mdf.project_id.isin(vsprojects)].reset_index(drop=True)

In [ ]:
# Submit the derived data to validatestaging

projects = list(set(vsmdf['project_id']))
vsdata = {}
for vspid in projects:
    print("Submitting data to project '{}'.".format(vspid))
    vsdata[vspid] = vsexp.submit_df(df=vsmdf.loc[vsmdf["project_id"]==vspid],project_id=vspid, chunk_size=1000)

In [ ]:
# Submit the derived data to staging

projects = list(set(smdf['project_id']))
sdata = {}
for spid in projects:
    print("Submitting data to project '{}'.".format(vspid))
    sdata[spid] = sexp.submit_df(df=smdf.loc[smdf["project_id"]==spid],project_id=spid, chunk_size=1000)